In [1]:
import pandas as pd
import numpy as np
import os
import glob
import pathlib
import re
import janome
import jaconv

In [2]:
p_dic = pathlib.Path('dic')

for i in p_dic.glob('*.txt'):
    with open (i, 'r', encoding='utf-8') as f:
        x = [i.replace('\n', '').split(':') for i in f.readlines()]
        
posi_nega_df = pd.DataFrame(x, columns = ['基本形', '読み', '品詞', 'スコア'])
posi_nega_df['読み'] = posi_nega_df['読み'].apply(lambda x : jaconv.hira2kata(x))
posi_nega_df = posi_nega_df[~posi_nega_df[['基本形', '読み', '品詞']].duplicated()]

In [3]:
posi_nega_df.head()

,基本形,読み,品詞,スコア
0,優れる,スグレル,動詞,1
1,良い,ヨイ,形容詞,0.999995
2,喜ぶ,ヨロコブ,動詞,0.999979
3,褒める,ホメル,動詞,0.999979
4,めでたい,メデタイ,形容詞,0.999645


In [4]:
posi_nega_df.shape

(55112, 4)

In [26]:
p_temp = pathlib.Path('text')
article_list = []
for p in p_temp.glob('**/*.txt'):
    if len(str(p).split('/')) >2:
        media = str(p).split('/')[1]
        file_name = str(p).split('/')[2]
    
        if file_name != 'LICENSE.txt':
            with open(p, 'r') as f:
                article = f.readlines()
                article = [re.sub(r'[\n \u3000]', '', i) for i in article]
            article_list.append([media, article[0], article[1], article[2], ''.join(article[3:])])

        else:
            continue
article_df = pd.DataFrame(article_list)
article_df.head()

,0,1,2,3,4


In [32]:
news_df.iloc[:5, 4]

0    今月8日、都内ホテルでは、総合格闘家・吉田秀彦の引退試合興行「ASTRA」の開催が発表された...
1    競馬界にひと足早い"春"が到来した。10日、ＪＲＡが2010年度の新規調教師９名、騎手免許試...
2    スノーボード男子ハーフパイプの国母和宏（東海大）が、バンクーバーへの移動中に空港で服装の乱れ...
3    バンクーバー五輪女子スノーボードクロス日本代表の藤森由香が公式練習中に風にあおられて転倒。頭...
4    バンクーバー五輪で日本勢初のメダル獲得となったスピードスケート男子500メートルの長島圭一郎...
Name: 4, dtype: object

In [38]:
news_df = article_df[article_df[0] == 'sports-watch'].reset_index(drop = True)

from janome.tokenizer import Tokenizer
from janome.analyzer import Analyzer
from janome.charfilter import *

i = 0

t = Tokenizer()
char_filters = [UnicodeNormalizeCharFilter()]
analyzer = Analyzer(char_filters=char_filters, tokenizer=t)

word_lists = []
for i, row in news_df.iterrows():
    for t in analyzer.analyze(row[4]):
        surf = t.surface
        base = t.base_form
        pos = t.part_of_speech
        reading = t.reading
        
        word_lists.append([i, surf, base, pos, reading])
word_df = pd.DataFrame(word_lists, columns=['ニュースNo.', '単語', '基本形', '品詞', '読み'])
word_df['品詞'] = word_df['品詞'].apply(lambda x : x.split(',')[0])

In [41]:
word_df.head()

,ニュースNo.,単語,基本形,品詞,読み
0,0,今月,今月,名詞,コンゲツ
1,0,8,8,名詞,*
2,0,日,日,名詞,ニチ
3,0,、,、,記号,、
4,0,都内,都内,名詞,トナイ


In [44]:
posi_nega_df.head()

,基本形,読み,品詞,スコア
0,優れる,スグレル,動詞,1
1,良い,ヨイ,形容詞,0.999995
2,喜ぶ,ヨロコブ,動詞,0.999979
3,褒める,ホメル,動詞,0.999979
4,めでたい,メデタイ,形容詞,0.999645


In [46]:
score_result = pd.merge(word_df, posi_nega_df, on=['基本形', '品詞', '読み'], how='left')

In [49]:
score_result.head(10)

,ニュースNo.,単語,基本形,品詞,読み,スコア
0,0,今月,今月,名詞,コンゲツ,-0.310828
1,0,8,8,名詞,*,NaN
2,0,日,日,名詞,ニチ,-0.903573
3,0,、,、,記号,、,NaN
4,0,都内,都内,名詞,トナイ,NaN
5,0,ホテル,ホテル,名詞,ホテル,-0.427329
6,0,で,で,助詞,デ,NaN
7,0,は,は,助詞,ハ,NaN
8,0,、,、,記号,、,NaN
9,0,総合,総合,名詞,ソウゴウ,-0.384064


In [53]:
result = []
for i in range(len(score_result['ニュースNo.'].unique())):
    temp_df = score_result[score_result['ニュースNo.']== i]
    text = ''.join(list(temp_df['単語']))
    score = temp_df['スコア'].astype(float).sum()
    score_r = score/temp_df['スコア'].astype(float).count()
    result.append([i, text, score, score_r])

pd.DataFrame(result, columns=['ニュースNo.', 'テキスト', '累計スコア', '標準化スコア']).sort_values(by='標準化スコア').reset_index(drop=True)

,ニュースNo.,テキスト,累計スコア,標準化スコア
0,341,フジテレビ「すぽると!」(2日深夜放送)では、チャンピオンズリーグ準決勝セカンドレグで、マン...,-33.127142,-0.637060
1,107,現在、日本中に大きな驚きと怒りをもたらしてる大相撲の野球賭博問題。旧態依然とした体質に加え、...,-41.976875,-0.626521
2,78,テレビ朝日「やべっちF.C.」(30日放送分)では、「日本代表緊急討論会」として、セルジオ越...,-61.373802,-0.619937
3,654,2日放送、日本テレビ系列「ダウンタウンDX」には、埼玉西武ライオンズの石井一久、中村剛也が出...,-33.803370,-0.614607
4,292,TBS「S1」(27日深夜)では、北海道日本ハムファイターズのエース・ダルビッシュ有と、プロ...,-53.934022,-0.612887
...,...,...,...,...
895,758,国内外を問わず、格闘技界には美女格闘家と呼ばれる選手たちが思いのほか多い。今でこそ第一線から...,-15.712103,-0.227712
896,533,10月27日に行われたドラフト会議で日本ハムの1位指名を受けた東海大・菅野智之投手。11月4...,-18.889230,-0.227581
897,395,2011年、日本生命セ・パ交流戦で優勝に輝いた福岡ソフトバンクホークス。22日には、交流戦M...,-14.983960,-0.211042
898,453,18日、サッカー女子日本代表の監督、コーチ、選手、スタッフ35名に国民栄誉賞が授与され、その...,-12.284698,-0.159542


In [54]:
data = pd.DataFrame(result, columns=['ニュースNo.', 'テキスト', '累計スコア', '標準化スコア']).sort_values(by='標準化スコア').reset_index(drop=True)

In [68]:
data.iloc[0, 1]

'フジテレビ「すぽると!」(2日深夜放送)では、チャンピオンズリーグ準決勝セカンドレグで、マンチェスター・ユナイテッドと対戦するシャルケ・内田篤人のインタビューを放送した。ファーストレグでは2-0と後れをとり、崖っぷちの一戦に臨むことになった内田。その試合について、「いつか点取られるなっていう気はしてたし、やっぱり強いなっていうのはありました。自分達がボールを回せないで走らされている分、後ろに負担がかかるので、まあ、しょうがないっすね。インテルはちょっと個人寄りな部分があったんで、こちらがしっかり準備できていれば、そんなに怖くないっていうのはあった」と振り返ると、「マンチェはかなり組織もしっかりしていて、成熟しているなというか、監督も長い時間やってるし、歴史を感じました」という感想を述べた。また、「ビビってるわけではないですけど、ノイアーが言ってたんですけど、(マンUを)尊重し過ぎた。日本語が適しているか分からないですけど」という内田は、大きなディスアドバンテージがあるセカンドレグに向け、「やってる自分達がそんなこと言ってたら、始まらないですから。嘘でも(攻略法は)あると言っておきます。というか、まあ、いるじゃないですか、一人。ラウールが何とかしてくれるかな。僕はシャルケでプレーしてなかったら、無理だなって思いますし、僕がマンUでプレーしてたら“早く諦めてくれ”って思いますし、でも、僕はシャルケでプレーしているんで、諦めないですし、こんなところで投げたくない。どんな厳しい状況であろうが、どういう大会だろうが関係なく、試合をやるからには負けたくない」と意気込んだ。'

In [69]:
data.iloc[1, 1]

'現在、日本中に大きな驚きと怒りをもたらしてる大相撲の野球賭博問題。旧態依然とした体質に加え、暴力団との繋がりも根が深く、4日、大関・琴光喜と大嶽親方には解雇処分が下された。TBS系「サンデー・ジャポン」(4日放送)では、テリー伊藤が本問題について言及、「博打ってこの程度の罪で済むのかと素人の人が思ってしまうのが危険」と警笛を鳴らすと、「本来ならば相撲協会がNHKに今回の名古屋場所は放送しないで下さいと言わなくてはいけないし、協会が体制を維持しようとする姿勢に違和感を感じる」と苦言を呈したばかりだった。そんな言葉を象徴するかのように、琴光喜は最も重い処分=除名ではないため、退職金が支給され、5日には一部のメディアが「満額なら5500万円」と伝えている。どこまでも、ぬるま湯体質が抜けきらず、進歩のない角界には、今度も多くの批判が寄せられそうだ。'

In [70]:
data.iloc[-1, 1]

'今シーズンは、最多勝、最優秀防御率、最多奪三振と22歳の史上最年少で投手三冠を達成し、沢村賞を獲得した広島東洋カープ・前田健太が、17日深夜放送のフジテレビ「すぽると!」に生出演を果たした。契約更改には、「そうですね。楽しみですね。いい成績だったので、気持ちよく一発でいきたいなと思います」と笑顔を見せた前田。「あんまり自分では実感がなくて、周りの人が喜んでくれて、やっと実感できるかな」と今シーズンの活躍を振り返った。また、大きく成績を向上させた要因については、「ピッチングのレベルも上がったんですけど、(昨シーズンの)14敗も本当に悔しかったし、何か変えたいなと思って。あと、ストレートが今年よくなったので、勝てるようになりました」と語る。そのストレートについては、「投げるコツみたいなものを掴んで、はやい球を投げれるようになって、投球に幅が出るようになりました。投げ方のコツですね。(3回目の先発でコツを掴んだと言われているが?)本当にその試合の一球だけで掴みました」と断言するのだった。'

In [71]:
data.iloc[-2, 1]

'18日、サッカー女子日本代表の監督、コーチ、選手、スタッフ35名に国民栄誉賞が授与され、その副賞には、日本の伝統工芸品でもある熊野筆(化粧筆)が進呈された。なでしこジャパン・丸山桂里奈は、報道陣から化粧筆が贈られた理由を訊かれるや、「ど・・・、なんなんですかね?なんだと思いますか?私もちょっと分からないんですけど」と素直に答え、国民栄誉賞の副賞とはいえ、化粧筆が贈られたことには困惑した表情を見せた。19日放送のTBS「みのもんたの朝ズバッ!」では、そんな丸山の母・慶子さんの談話を紹介。「国民栄誉賞受賞者を育てる秘訣」を訊かれると、「特別こういう育て方をしたというのは、特にないんですけどね」と答えた慶子さんだったが、「やっぱり愛情は注いでました。あと、隠し事はしないで何でも話し合える、そういうあれはありました」と語る。千葉県民栄誉賞、東京都栄誉賞など、なでしこジャパントップの9つの賞を受賞した丸山は、着うたで歌手デビューをするなど、タレント並みの活動も行うが、VTRの最後には慶子さんも「国民栄誉賞トゥーユー」という歌で娘を祝福した。'